In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, RectBivariateSpline
from scipy.optimize import minimize
import warnings
import import_ipynb
from scipy.stats import norm

warnings.filterwarnings('ignore')

from one_iv_computation import get_spot_and_risk_free_rate 
IV_surface_calls = pd.read_pickle('IV_surface_calls.pkl')
IV_surface_puts = pd.read_pickle('IV_surface_puts.pkl')


current SPY Price: $694.0700073242188
Risk-Free Rate: 0.041710000038146976 (4.171000003814697%)
Total calls: 865 (liquid: 405)

sample call options:
    strike    bid    ask  lastPrice  volume  impliedVolatility      expiry  \
1    640.0  52.62  55.41      54.37   280.0           0.630497  2026-01-12   
2    645.0  47.64  50.42      49.88   214.0           0.584721  2026-01-12   
3    650.0  42.64  45.38      44.52    23.0           0.533452  2026-01-12   
9    664.0  28.65  31.43      30.64    27.0           0.403204  2026-01-12   
10   665.0  27.64  30.43      29.05    11.0           0.393317  2026-01-12   

    dte     mid  
1     1  54.015  
2     1  49.030  
3     1  44.010  
9     1  30.040  
10    1  29.035  
successfully computed IVs, moving onto analysis...


comparison (calls)
successfully computed: 361 / 405
mean difference: -0.003363726464735645
std difference: 0.03901073394810251
max (absolute) diff: 0.2684701713071319

comparison (puts)
successfully computed: 568 / 568
me

In [2]:
# some helpers

def black_scholes(S, K, T, r, sigma, option_type):
    # compute black scholes options price 
    if T <= 0:
        return max(S - K, 0)
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if option_type=='put':
        return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    else:
        return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def iv_to_price(iv_surface, S, r, option_type='call'):
    # IV surface to price surface 
    prices = iv_surface.copy()
    
    for exp in iv_surface.columns:
        T = exp  # Assuming columns are time to expiration in years
        for K in iv_surface.index:
            sigma = iv_surface.loc[K, exp]
            if pd.notna(sigma) and sigma > 0:
                if option_type == 'call':
                    prices.loc[K, exp] = black_scholes(S, K, T, r, sigma,'call')
                else:
                    prices.loc[K, exp] = black_scholes(S, K, T, r, sigma,'put')
            else:
                prices.loc[K, exp] = np.nan
    
    return prices


In [3]:
def detect_calendar_arbitrage(price_surface, tolerance=0.01):
    """
    arb #1: longer-dated options must be worth more. if not, save to a dataframe. 
    """
    violations = []
    expirations = sorted(price_surface.columns)
    
    for strike in price_surface.index:
        for i in range(len(expirations) - 1):
            t1, t2 = expirations[i], expirations[i+1]
            p1 = price_surface.loc[strike, t1]
            p2 = price_surface.loc[strike, t2]
            
            if pd.notna(p1) and pd.notna(p2):
                if p1 > p2 + tolerance:
                    violations.append({
                        'strike': strike,
                        'short_exp': t1,
                        'long_exp': t2,
                        'short_price': p1,
                        'long_price': p2,
                        'violation': p1 - p2
                    })
    
    return pd.DataFrame(violations)

def visualize_calendar_arbitrage(violations, price_surface):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # heatmap 
    if len(violations) > 0:
        pivot = violations.pivot_table(
            values='violation',
            index='strike',
            columns='short_exp',
            aggfunc='sum'
        )
        sns.heatmap(pivot, ax=axes[0], cmap='Reds', annot=True, fmt='.2f')
        axes[0].set_title('Calendar Arbitrage Violations by Strike/Expiration')
        axes[0].set_ylabel('Strike')
        axes[0].set_xlabel('Expiration')
    else:
        axes[0].text(0.5, 0.5, 'No Violations', ha='center', va='center')
        axes[0].set_title('Calendar Arbitrage Violations')
    
    # price surface
    for strike in price_surface.index[::5]:  # sample strikes
        prices = price_surface.loc[strike, :].dropna()
        axes[1].plot(prices.index, prices.values, marker='o', label=f'K={strike:.0f}')
    
    axes[1].set_title('Option Prices Across Expirations')
    axes[1].set_xlabel('Time to Expiration')
    axes[1].set_ylabel('Option Price')
    axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig


In [4]:
def detect_vertical_arbitrage(price_surface, tolerance=0.01):
    """
    C(K1) >= C(K2) for K1 < K2
    C(K1) - C(K2) <= K2 - K1
    """
    violations = []
    
    for exp in price_surface.columns:
        prices = price_surface[exp].dropna().sort_index()
        strikes = prices.index
        
        for i in range(len(strikes) - 1):
            k1, k2 = strikes[i], strikes[i+1]
            p1, p2 = prices[k1], prices[k2]
            
            if p2 > p1 + tolerance:
                violations.append({
                    'expiration': exp,
                    'strike_low': k1,
                    'strike_high': k2,
                    'price_low': p1,
                    'price_high': p2,
                    'violation_type': 'monotonicity',
                    'violation': p2 - p1
                })
            
            spread_value = p1 - p2
            max_spread = k2 - k1
            if spread_value > max_spread + tolerance:
                violations.append({
                    'expiration': exp,
                    'strike_low': k1,
                    'strike_high': k2,
                    'price_low': p1,
                    'price_high': p2,
                    'violation_type': 'spread_bound',
                    'violation': spread_value - max_spread
                })
    
    return pd.DataFrame(violations)

def visualize_vertical_arbitrage(violations, price_surface):
    n_exp = len(price_surface.columns)
    n_cols = min(3, n_exp)
    n_rows = (n_exp + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(6*n_cols, 5*n_rows))
    if n_exp == 1:
        axes = np.array([axes])
    axes = axes.flatten()
    
    for idx, exp in enumerate(price_surface.columns):
        ax = axes[idx]
        prices = price_surface[exp].dropna().sort_index()
        
        # prices
        ax.plot(prices.index, prices.values, 'b-o', label='Prices', linewidth=2)
        
        #  violations
        exp_violations = violations[violations['expiration'] == exp]
        for _, viol in exp_violations.iterrows():
            if viol['violation_type'] == 'monotonicity':
                ax.plot([viol['strike_low'], viol['strike_high']], 
                       [viol['price_low'], viol['price_high']], 
                       'r-o', linewidth=3, markersize=8, label='Monotonicity')
            elif viol['violation_type'] == 'spread_bound':
                ax.plot([viol['strike_low'], viol['strike_high']], 
                       [viol['price_low'], viol['price_high']], 
                       'orange', linewidth=3, markersize=8, label='Spread Bound')
        
        ax.set_title(f'Expiration: {exp:.3f} years')
        ax.set_xlabel('Strike')
        ax.set_ylabel('Option Price')
        ax.grid(True, alpha=0.3)
        
        # remove duplicates labels
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        ax.legend(by_label.values(), by_label.keys())
    
    for idx in range(n_exp, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    return fig


In [5]:
def detect_butterfly_arbitrage(price_surface, tolerance=0.01):
    """
    For K1 < K2 < K3: C(K1) - 2*C(K2) + C(K3) >= 0
    """
    violations = []
    
    for exp in price_surface.columns:
        prices = price_surface[exp].dropna().sort_index()
        strikes = prices.index
        
        for i in range(len(strikes) - 2):
            k1, k2, k3 = strikes[i], strikes[i+1], strikes[i+2]
            p1, p2, p3 = prices[k1], prices[k2], prices[k3]
            
            butterfly = p1 - 2*p2 + p3
            
            h1 = k2 - k1
            h2 = k3 - k2
            
            if butterfly < -tolerance:
                violations.append({
                    'expiration': exp,
                    'strike_low': k1,
                    'strike_mid': k2,
                    'strike_high': k3,
                    'price_low': p1,
                    'price_mid': p2,
                    'price_high': p3,
                    'butterfly_value': butterfly,
                })
    
    return pd.DataFrame(violations)

def visualize_butterfly_arbitrage(violations, price_surface):
    n_exp = len(price_surface.columns)
    n_cols = min(3, n_exp)
    n_rows = (n_exp + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(6*n_cols, 5*n_rows))
    if n_exp == 1:
        axes = np.array([axes])
    axes = axes.flatten()
    
    for idx, exp in enumerate(price_surface.columns):
        ax = axes[idx]
        prices = price_surface[exp].dropna().sort_index()
        strikes = prices.index
        
        butterfly_values = []
        butterfly_strikes = []
        for i in range(len(strikes) - 2):
            k1, k2, k3 = strikes[i], strikes[i+1], strikes[i+2]
            p1, p2, p3 = prices[k1], prices[k2], prices[k3]
            butterfly = p1 - 2*p2 + p3
            butterfly_values.append(butterfly)
            butterfly_strikes.append(k2)
        
        ax.plot(butterfly_strikes, butterfly_values, 'b-o', linewidth=2)
        ax.axhline(y=0, color='k', linestyle='--', alpha=0.5)
        
        exp_violations = violations[violations['expiration'] == exp]
        if len(exp_violations) > 0:
            ax.scatter(exp_violations['strike_mid'], 
                      exp_violations['butterfly_value'],
                      color='red', s=100, zorder=5, label='Violations')
        
        ax.set_title(f'Butterfly Convexity - Exp: {exp:.3f} years')
        ax.set_xlabel('Strike (Middle)')
        ax.set_ylabel('Butterfly Value (C₁ - 2C₂ + C₃)')
        ax.grid(True, alpha=0.3)
        ax.legend()
    
    for idx in range(n_exp, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    return fig


In [6]:
def detect_put_call_parity_violations(call_prices, put_prices, S, r, tolerance=0.05):
    """
    Parameters:
    - call_prices: DataFrame of call prices (strikes x expirations)
    - put_prices: DataFrame of put prices (strikes x expirations)
    - S: current stock price
    - r: risk-free rate
    - tolerance: tolerance for violations (as fraction of stock price)
    """
    violations = []
    
    common_strikes = call_prices.index.intersection(put_prices.index)
    common_expirations = call_prices.columns.intersection(put_prices.columns)
    
    for exp in common_expirations:
        T = exp
        for strike in common_strikes:
            call_price = call_prices.loc[strike, exp]
            put_price = put_prices.loc[strike, exp]
            
            if pd.notna(call_price) and pd.notna(put_price):
                # Put-call parity
                lhs = call_price - put_price
                rhs = S - strike * np.exp(-r * T)
                violation = lhs - rhs
                
                if abs(violation) > tolerance * S:
                    violations.append({
                        'strike': strike,
                        'expiration': exp,
                        'call_price': call_price,
                        'put_price': put_price,
                        'parity_lhs': lhs,
                        'parity_rhs': rhs,
                        'violation': violation,
                        'violation_pct': 100 * violation / S
                    })
    
    return pd.DataFrame(violations)

def visualize_put_call_parity(violations, call_prices, put_prices, S, r):
    n_exp = min(6, len(call_prices.columns))  # Show up to 6 expirations
    n_cols = 3
    n_rows = (n_exp + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(6*n_cols, 5*n_rows))
    if n_exp == 1:
        axes = np.array([axes])
    axes = axes.flatten()
    
    expirations = sorted(call_prices.columns)[:n_exp]
    
    for idx, exp in enumerate(expirations):
        ax = axes[idx]
        T = exp
        
        common_strikes = call_prices.index.intersection(put_prices.index)
        calls = call_prices.loc[common_strikes, exp].dropna()
        puts = put_prices.loc[common_strikes, exp].dropna()
        
        common = calls.index.intersection(puts.index)
        lhs = calls[common] - puts[common]
        rhs = S - common.values * np.exp(-r * T)
        
        ax.plot(common, lhs.values, 'b-o', label='C - P (LHS)', linewidth=2)
        ax.plot(common, rhs, 'g--', label='S - K*e^(-rT) (RHS)', linewidth=2)
        
        exp_violations = violations[violations['expiration'] == exp]
        if len(exp_violations) > 0:
            ax.scatter(exp_violations['strike'], 
                      exp_violations['parity_lhs'],
                      color='red', s=100, zorder=5, label='Violations')
        
        ax.set_title(f'Put-Call Parity - Exp: {exp:.3f} years')
        ax.set_xlabel('Strike')
        ax.set_ylabel('Value')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    for idx in range(n_exp, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    return fig


In [7]:
find_arbitrage = False 
if find_arbitrage:
    S,r = get_spot_and_risk_free_rate()
    
    # convert IV to price surfaces 
    print("Converting IV to prices...")
    call_prices = iv_to_price(IV_surface_calls, S, r, option_type='call')
    put_prices = iv_to_price(IV_surface_puts, S, r, option_type='put')
    
    # calendar arbitrage 
    print("\n calendar arbitrage (calls)") 
    print("\n calls ")
    calendar_violations_calls = detect_calendar_arbitrage(call_prices)
    print(f"# violations found: {len(calendar_violations_calls)}")
    if len(calendar_violations_calls) > 0:
        print(f" max violation: ${calendar_violations_calls['violation'].max()}")
        print(f" total violation: ${calendar_violations_calls['violation'].sum()}")
        fig_cal_calls = visualize_calendar_arbitrage(calendar_violations_calls, call_prices)
        plt.show()
    
    print("\n calendar arbitrage (puts)")
    calendar_violations_puts = detect_calendar_arbitrage(put_prices)
    print(f" violations found: {len(calendar_violations_puts)}")
    if len(calendar_violations_puts) > 0:
        print(f" max violation: ${calendar_violations_puts['violation'].max()}")
        fig_cal_puts = visualize_calendar_arbitrage(calendar_violations_puts, put_prices)
        plt.show()
    
    # vertical spread 
    print("\n vertical spread arbitrage (calls)")
    vertical_violations_calls = detect_vertical_arbitrage(call_prices)
    print(f"violations found: {len(vertical_violations_calls)}")
    if len(vertical_violations_calls) > 0:
        print(f"monotonicity violations: {len(vertical_violations_calls[vertical_violations_calls['violation_type']=='monotonicity'])}")
        print(f"spread bound violations: {len(vertical_violations_calls[vertical_violations_calls['violation_type']=='spread_bound'])}")
        fig_vert_calls = visualize_vertical_arbitrage(vertical_violations_calls, call_prices)
        plt.show()
    
    print("\n vertical spread arbitrage (puts)")
    vertical_violations_puts = detect_vertical_arbitrage(put_prices)
    print(f"violations found: {len(vertical_violations_puts)}")
    if len(vertical_violations_puts) > 0:
        print(f"monotonicity violations: {len(vertical_violations_puts[vertical_violations_puts['violation_type']=='monotonicity'])}")
        print(f"spread bound violations: {len(vertical_violations_puts[vertical_violations_puts['violation_type']=='spread_bound'])}")
        fig_vert_puts = visualize_vertical_arbitrage(vertical_violations_puts, put_prices)
        plt.show()
    
    # butterfly 
    print("\n butterfly arbitrage (calls)")
    butterfly_violations_calls = detect_butterfly_arbitrage(call_prices)
    print(f"violations found: {len(butterfly_violations_calls)}")
    if len(butterfly_violations_calls) > 0:
        print(f"max violation: {butterfly_violations_calls['butterfly_value'].min()}")
        fig_fly_calls = visualize_butterfly_arbitrage(butterfly_violations_calls, call_prices)
        plt.show()
    
    print("\n butterfly arbitrage (puts)")
    butterfly_violations_puts = detect_butterfly_arbitrage(put_prices)
    print(f"violations found: {len(butterfly_violations_puts)}")
    if len(butterfly_violations_puts) > 0:
        print(f"max violation: {butterfly_violations_puts['butterfly_value'].min()}")
        fig_fly_puts = visualize_butterfly_arbitrage(butterfly_violations_puts, put_prices)
        plt.show()
    
    # put call parity
    print("\n put-call parity")
    pcp_violations = detect_put_call_parity_violations(call_prices, put_prices, S, r)
    print(f"violations found: {len(pcp_violations)}")
    if len(pcp_violations) > 0:
        print(f"   Max violation: ${abs(pcp_violations['violation']).max()} ({abs(pcp_violations['violation_pct']).max()}%)")
        print(f"   Mean violation: ${abs(pcp_violations['violation']).mean()} ({abs(pcp_violations['violation_pct']).mean()}%)")
        fig_pcp = visualize_put_call_parity(pcp_violations, call_prices, put_prices, S, r)
        plt.show()
    
    print("summary!")
    total_violations = (len(calendar_violations_calls) + len(calendar_violations_puts) +
                       len(vertical_violations_calls) + len(vertical_violations_puts) +
                       len(butterfly_violations_calls) + len(butterfly_violations_puts) +
                       len(pcp_violations))
    # Save violation dataframes for further analysis
    print("\nViolation DataFrames saved as:")
    print("  - calendar_violations_calls")
    print("  - calendar_violations_puts")
    print("  - vertical_violations_calls")
    print("  - vertical_violations_puts")
    print("  - butterfly_violations_calls")
    print("  - butterfly_violations_puts")
    print("  - pcp_violations")